# Data Science Unit 4 Sprint Challenge 1 — Tree Ensembles

### Chicago Food Inspections

For this Sprint Challenge, you'll use a dataset with information from inspections of restaurants and other food establishments in Chicago from January 1, 2010 to the present. 

[See this PDF](https://data.cityofchicago.org/api/assets/BAD5301B-681A-4202-9D25-51B2CAE672FF) for descriptions of the data elements included in this dataset.

According to [Chicago Department of Public Health — Food Protection Services](https://www.chicago.gov/city/en/depts/cdph/provdrs/healthy_restaurants/svcs/food-protection-services.html), "Chicago is home to 16,000 food establishments like restaurants, grocery stores, bakeries, wholesalers, lunchrooms, mobile food vendors and more. Our business is food safety and sanitation with one goal, to prevent the spread of food-borne disease. We do this by inspecting food businesses, responding to complaints and food recalls." 

#### Your challenge: Predict whether inspections failed

The target is the `Fail` column.

- When the food establishment failed the inspection, the target is `1`.
- When the establishment passed, the target is `0`.

#### Run this cell to load the data:

In [78]:
import pandas as pd
import numpy as np
import category_encoders as ce
import xgboost as xgb

train_url = 'https://drive.google.com/uc?export=download&id=13_tP9JpLcZHSPVpWcua4t2rY44K_s4H5'
test_url  = 'https://drive.google.com/uc?export=download&id=1GkDHjsiGrzOXoF_xcYjdzBTSjOIi3g5a'

train = pd.read_csv(train_url)
test  = pd.read_csv(test_url)

X_train_ = train.drop(['Fail'], axis=1)
X_test_ = test.drop(['Fail'], axis=1)
y_train = train.Fail
y_test = test.Fail

assert train.shape == (51916, 17)
assert test.shape  == (17306, 17)

### Part 1: Preprocessing

You may choose which features you want to use, and whether/how you will preprocess them. You may use any tools and techniques for categorical encoding. (Pandas, category_encoders, sklearn.preprocessing, or any other library.)

_To earn a score of 3 for this part, engineer new features, and use any alternative categorical encoding instead of One-Hot or Ordinal/Label encoding._

### Part 2: Modeling

Fit a Random Forest or Gradient Boosting model with the train set. (You may use scikit-learn, xgboost, or any other library.) Use cross-validation to estimate an ROC AUC validation score.

Use your model to predict probabilities for the test set. Get an ROC AUC test score >= 0.60.

_To earn a score of 3 for this part, get an ROC AUC test score >= 0.70._


### Part 3: Visualization

Make one visualization for model interpretation. (You may use any libraries.) Choose one of these types:
- Feature Importances
- Permutation Importances
- Partial Dependence Plot

_To earn a score of 3 for this part, make at least two of these visualization types._

In [79]:
train.head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Violations,Latitude,Longitude,Location,Fail
0,2088270,"TOM YUM RICE & NOODLE, INC.",TOM YUM CAFE,2354911.0,Restaurant,Risk 1 (High),608 W BARRY,CHICAGO,IL,60657.0,2017-09-15T00:00:00,Canvass,3. POTENTIALLY HAZARDOUS FOOD MEETS TEMPERATUR...,41.938007,-87.644755,"{'longitude': '-87.6447545707008', 'latitude':...",1
1,555268,FILLING STATION & CONVENIENCE STORE,FILLING STATION & CONVENIENCE STORE,1044901.0,Grocery Store,Risk 3 (Low),6646-6658 S WESTERN AVE,CHICAGO,IL,60636.0,2011-10-20T00:00:00,Complaint Re-Inspection,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.772402,-87.683603,"{'longitude': '-87.68360273081268', 'latitude'...",0
2,1751394,A P DELI,A P DELI,47405.0,Restaurant,Risk 1 (High),2025 E 75TH ST,CHICAGO,IL,60649.0,2016-04-05T00:00:00,Canvass Re-Inspection,"35. WALLS, CEILINGS, ATTACHED EQUIPMENT CONSTR...",41.758779,-87.575054,"{'longitude': '-87.57505446746121', 'latitude'...",0
3,1763905,FRANK'S CHICAGO SHRIMP HOUSE,FRANK'S CHICAGO SHRIMP HOUSE,6414.0,Restaurant,Risk 2 (Medium),4459 S ARCHER AVE,CHICAGO,IL,60632.0,2016-04-29T00:00:00,Canvass,38. VENTILATION: ROOMS AND EQUIPMENT VENTED AS...,41.812181,-87.707125,"{'longitude': '-87.70712481334274', 'latitude'...",0
4,453326,MORRILL,MORRILL,24571.0,School,Risk 1 (High),6011 S Rockwell (2600W) AVE,CHICAGO,IL,60629.0,2011-01-10T00:00:00,Canvass,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO...",NaN,NaN,NaN,0


In [80]:
train.dtypes

Inspection ID        int64
DBA Name            object
AKA Name            object
License #          float64
Facility Type       object
Risk                object
Address             object
City                object
State               object
Zip                float64
Inspection Date     object
Inspection Type     object
Violations          object
Latitude           float64
Longitude          float64
Location            object
Fail                 int64
dtype: object

In [81]:
train.describe()

,Inspection ID,License #,Zip,Latitude,Longitude,Fail
count,5.191600e+04,5.191100e+04,51890.000000,51718.000000,51718.000000,51916.000000
mean,1.330265e+06,1.549491e+06,60628.834669,41.879568,-87.676663,0.258610
std,6.202664e+05,9.018283e+05,22.736963,0.081126,0.059509,0.437875
min,4.425000e+04,0.000000e+00,60007.000000,41.644670,-87.914428,0.000000
25%,6.796205e+05,1.140745e+06,60614.000000,41.830296,-87.707857,0.000000
50%,1.424226e+06,1.958619e+06,60625.000000,41.890166,-87.666524,0.000000
75%,1.946952e+06,2.215507e+06,60643.000000,41.939167,-87.634461,1.000000
max,2.279709e+06,8.700606e+06,60827.000000,42.021064,-87.525094,1.000000


In [82]:
X_train_.isna().sum()

Inspection ID         0
DBA Name              0
AKA Name            623
License #             5
Facility Type       224
Risk                 12
Address               0
City                 53
State                10
Zip                  26
Inspection Date       0
Inspection Type       1
Violations         9655
Latitude            198
Longitude           198
Location            198
dtype: int64

In [83]:
todrop1 = ['Inspection ID', 'AKA Name', 'License #', 'Location']

def wrangle(dat): 
    import datetime as dt
    assigns = {
        **{'Violations': dat.Violations.fillna('NO_VIOLATIONS'), 
          'Facility Type': dat['Facility Type'].fillna('NOT_APPLICABLE'), 
          'Inspection Date': pd.to_datetime(dat['Inspection Date'], 
                                            infer_datetime_format=True
                                           ).apply(dt.datetime.toordinal)}
    }
    
    return dat.assign(**assigns).drop(todrop1, axis=1)

X_train = wrangle(X_train_)
X_test = wrangle(X_test_)

In [84]:
X_train.isna().sum()

DBA Name             0
Facility Type        0
Risk                12
Address              0
City                53
State               10
Zip                 26
Inspection Date      0
Inspection Type      1
Violations           0
Latitude           198
Longitude          198
dtype: int64

In [85]:
print(f'The target is unbalanced target by a factor of {y_train.value_counts()[0] / y_train.value_counts()[1]} favoring `0`. \n\n')

cats = X_train.select_dtypes(include='object').columns
nums = X_train.select_dtypes(exclude='object').columns

print(f"We have {len(cats)} categoricals and {len(nums)} numerics. \n")

cards = {n: X_train[n].value_counts().shape[0] for n in cats}

def report_cards(c): 
    l = [f"\tFeature {k} has cardinality {v}. " for k,v in c.items()]
    return '\n'.join(l)

print(report_cards(cards))

The target is unbalanced target by a factor of 2.866825562341725 favoring `0`. 


We have 8 categoricals and 4 numerics. 

	Feature DBA Name has cardinality 17049. 
	Feature Facility Type has cardinality 330. 
	Feature Risk has cardinality 3. 
	Feature Address has cardinality 13954. 
	Feature City has cardinality 39. 
	Feature State has cardinality 1. 
	Feature Inspection Type has cardinality 51. 
	Feature Violations has cardinality 42165. 


In [86]:
## We're going to encode "high" and "low" cardinality featuers differently, depending on some meaning of "high"/"low"

thresh=2**12
low = [k for k in cards.keys() if cards[k]<=thresh]
high = [k for k in cards.keys() if cards[k]>thresh]

assert len(low) + len(high) == len(cards.keys())

# send lows to binary encoder and send highs to target encoder. 
low, high

['Facility Type', 'Risk', 'City', 'State', 'Inspection Type']

In [ ]:
def encode(dat, low=low, high=high, targ=y_train): 
    assert [x=='object' for x in dat.dtypes]

    bl = ce.BinaryEncoder()
    th = ce.TargetEncoder()
    
    bl_df = bl.fit_transform(dat[low])
    th_df = th.fit_transform(dat[high], targ)
    return bl_df, th_df

encode(X_train[cats])[0].head()